In [36]:
from __future__ import division
import os,time,pdb
import numpy as np

from insight_hds.preprocessing import datasets
reload(datasets) 

<module 'insight_hds.preprocessing.datasets' from '/home/bujan/storage/development/insight_hds/preprocessing/datasets.py'>

## Get data from subreddits

In [37]:
subreddits = ['schizophrenia', 'depression', 'bipolar']

In [38]:
X, y = datasets.scrape_comments(subreddits)


998 comments from subreddit: schizophrenia fetched!

953 comments from subreddit: depression fetched!

999 comments from subreddit: bipolar fetched!


In [39]:
X.shape

(2950, 9972)